In [ ]:
import scanpy as sc
import pandas as pd
import anndata as ad

In [ ]:
import mygene

In [ ]:
import decoupler as dc

In [ ]:
# Python packages
#import scanpy as sc
#import scvi
#import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt
#import matplotlib
#import seaborn as sns
#import  decoupler as dc

In [ ]:
path = "/data/scratch/kvalem/projects/2021/microbial-metabolites/preprocessing/40_single-cell-sorted-cd8/50_scrna_seq_by_sample/test/work/e7/055b59d0a151a805841cf4d86020b8"

In [ ]:
adata_solo = sc.read_h5ad(f"{path}/adata_nodoublet.h5ad")

In [ ]:
adata_solo

In [ ]:
sc.tl.leiden(adata_solo, key_added="leiden", resolution=0.3)

In [ ]:
sc.tl.leiden(adata_solo, resolution=1, key_added="leiden_2")

In [ ]:
sc.tl.leiden(adata_solo, resolution=0.7, key_added="leiden_3")

In [ ]:
def update_columns(row):
   
    if "ICI" not in row['sample_id']:
        row['origin'] = "colon"
    else:
         row['origin'] = "til"
        
    return row

# Apply the function to each row in the DataFrame
adata_solo.obs = adata_solo.obs.apply(update_columns, axis=1)

In [ ]:
def update_columns(row):
   
    if "ICI1"  in row['sample_id']:
        row['batch_id'] = "ICI1"
    elif "ICI2" in row['sample_id']:
         row['batch_id'] = "ICI2"
    elif row['sample_id'] == "10mix1":
        row['batch_id'] = "1"
    elif row['sample_id'] == "10mix2":
         row['batch_id'] = "2"
    elif row['sample_id'] == "11mix1":
        row['batch_id'] = "1"
    elif row['sample_id'] == "11mix2":
         row['batch_id'] = "2"
    elif row['sample_id'] == "GF1":
        row['batch_id'] = "1"
  
        
        
    return row

# Apply the function to each row in the DataFrame
adata_solo.obs = adata_solo.obs.apply(update_columns, axis=1)

In [ ]:
def update_columns(row):
   
    if "10mix-ICI1"  in row['sample_id']:
        row['ici_therapy'] = "Yes"
    elif "10mix-ICI2" in row['sample_id']:
        row['ici_therapy'] = "Yes"
    elif "11mix-ICI1" in row['sample_id']:
        row['ici_therapy'] = "Yes"
    elif "11mix-ICI2" in row['sample_id']:
        row['ici_therapy'] = "Yes"
    elif "GF-ICI2-plus" in row['sample_id']:
        row['ici_therapy'] = "Yes"
    elif "GF-ICI1-plus" in row['sample_id']:
        row['ici_therapy'] = "Yes"
    elif "GF-ICI1" in row['sample_id']:
        row['ici_therapy'] = "No"
    elif "GF-ICI2" in row['sample_id']:
        row['ici_therapy'] = "No"

  
        
    return row

# Apply the function to each row in the DataFrame
adata_solo.obs = adata_solo.obs.apply(update_columns, axis=1)

In [ ]:
def update_columns(row):
   
    if row['sample_id'] == "10mix1":
        row['condition'] = "10mix"
    elif row['sample_id'] == "10mix2":
         row['condition'] = "10mix"
    elif row['sample_id'] == "11mix1":
         row['condition'] = "11mix"
    elif row['sample_id'] == "11mix2":
         row['condition'] = "11mix"
    elif row['sample_id'] == "GF1":
         row['condition'] = "GF"
    elif row['sample_id'] == "GF2":
         row['condition'] = "GF"
    elif row['sample_id'] == "GF-ICI2-plus":
         row['condition'] = "GF-plus"
    elif row['sample_id'] == "GF-ICI1-plus":
         row['condition'] = "GF-plus"
    elif row['sample_id'] == "GF-ICI2":
         row['condition'] = "GF"
    elif row['sample_id'] == "GF-ICI1":
         row['condition'] = "GF"
    elif row['sample_id'] == "10mix-ICI1":
         row['condition'] = "10mix"
    elif row['sample_id'] == "10mix-ICI2":
         row['condition'] = "10mix"
    elif row['sample_id'] == "11mix-ICI1":
         row['condition'] = "11mix"
    elif row['sample_id'] == "11mix-ICI2":
         row['condition'] = "11mix"
    
    
        
        
    return row

# Apply the function to each row in the DataFrame
adata_solo.obs = adata_solo.obs.apply(update_columns, axis=1)

In [ ]:
adata_solo.obs[["batch_id","sample_id","origin","condition","ici_therapy"]].head(50000)

In [ ]:
list_var = adata_solo.var.index.unique()

In [ ]:
len(list_var)

In [ ]:
mg = mygene.MyGeneInfo()
ginfo = mg.querymany(list_var, scopes='ensembl.gene')

In [ ]:
df = pd.DataFrame(ginfo)

In [ ]:
df.drop_duplicates(subset='query', keep='last', inplace=True)

In [ ]:
df

In [ ]:
adata_solo.var["query"] = adata_solo.var.index

In [ ]:
adata_solo.var = df.merge(adata_solo.var, on='query', how='left')

In [ ]:
adata_solo.var['symbol'] = adata_solo.var.apply(lambda row: row['query'] if pd.isna(row['symbol']) else row['symbol'], axis=1)


In [ ]:
adata_solo.var

In [ ]:
sc.pl.umap(adata_solo, color = "leiden")

In [ ]:
sc.pl.umap(adata_solo, color = "sample_id")

In [ ]:
sc.pl.umap(adata_solo, color = "condition")

In [ ]:
sc.pl.umap(adata_solo, color = "ici_therapy")

In [ ]:
sc.pl.umap(adata_solo, color = "origin")

In [ ]:
adata_solo.var.index.rename(None)

In [ ]:
# marker genes 
marker_genes= pd.read_csv("/data/scratch/kvalem/projects/2021/microbial-metabolites/preprocessing/40_single-cell-sorted-cd8/40_scrna_seq/bin/notebooks/gene_markers.csv")

In [ ]:
marker_genes = marker_genes.drop_duplicates(subset=['symbol'])

In [ ]:
marker_genes = {"Tex": ["Cxcr6","Gzmb","Havcr2","Ctla4"],"Tisg":["Isg15","Ifit3","Ifit1","Stat1"],"Tn":["Ltb","Sell","Lef1","Nosip"],"Trm":["Itga1","Cpne7","Txnip","Jaml"],"Tsen":["Plcg2","Igkc","Klrk1","Slamf7"],"Tsrt":["Hspa1b", "Hspa1a","Dnajb1","Ubc"]}

- Tn - Naive T cells
- Tex - exhausted CD8+ T cell
- Tisg - interferon (IFN) response T cell
- Trm - Resident Memory T cells
- Tsen - senescent-like T cells
- Tstr - stress response T cell (identified a unique stress response state, TSTR, characterized by heat shock gene expression. TSTR cells are detectable in situ in the tumor microenvironment across various cancer types,)

- Teff - Effector T (Teff) cells
- t-Teff - transitional effector T (Teff) cells
- pTex - precursor exhausted
- Tcm - T CM, central memory T cells;


In [ ]:
marker_genes

In [ ]:
cd8t_cts = [
    "Tex",
    "Tisg",
    "Tn",
    "Trm",
    "Tsen",
    "Tsrt"
]

In [ ]:
#marker_genes = {
#    ct: [m for m in ct_markers if m in adata_solo.var.index]
#    for ct, ct_markers in marker_genes.items()
#    if ct in cd8t_cts
#}

In [ ]:
marker_genes

In [ ]:
#sc.pl.dotplot(
 #   adata_solo,
 #   groupby="leiden",
 #   var_names=marker_genes,
 #   gene_symbol = "symbol"
#)

In [ ]:
adata_solo.var

In [ ]:
adata_solo

In [ ]:
sc.pl.umap(
        adata_solo,
        color=["Casp7","Casp3","Fas","Casp9","Bcl2"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
adata_annot_colon

In [ ]:
adata_annot = adata_solo.copy()

In [ ]:
values_to_keep = ['10mix1', '10mix2', '11mix1', '11mix2',"GF1"]

adata_annot_colon = adata_annot[adata_annot.obs["sample_id"].isin(values_to_keep)]


In [ ]:
values_to_keep = ['11mix-ICI1','GF-ICI1-plus','10mix-ICI1', '10mix-ICI2', 'GF-ICI2-plus', '11mix-ICI2','11mix2', 'GF-ICI1', 'GF-ICI2']

adata_annot_tumor = adata_annot[adata_annot.obs["sample_id"].isin(values_to_keep)]

In [ ]:
del adata_annot_colon.uns["neighbors"]
del adata_annot_colon.obs["leiden_3"]
del adata_annot_colon.obs["leiden_2"]
del adata_annot_colon.obs["leiden"]

In [ ]:
del adata_annot_tumor.uns["neighbors"]
del adata_annot_tumor.obs["leiden_3"]
del adata_annot_tumor.obs["leiden_2"]
del adata_annot_tumor.obs["leiden"]

In [ ]:
sc.pp.neighbors(adata_annot_colon)

In [ ]:
sc.tl.umap(adata_annot_colon)

In [ ]:

sc.tl.leiden(adata_annot_colon, resolution=0.7, key_added="leiden_3")

In [ ]:

sc.tl.leiden(adata_annot_colon,  key_added="leiden")

In [ ]:

sc.tl.leiden(adata_annot_colon, resolution=0.3, key_added="leiden_2")

In [ ]:
sc.pl.umap(adata_annot_colon, color = "leiden_3",legend_loc="on data")

In [ ]:
sc.pl.umap(adata_annot_colon, color = "leiden",legend_loc="on data")

In [ ]:
sc.pl.umap(adata_annot_colon, color = "leiden_2",legend_loc="on data")

In [ ]:
#tn 7.9.11.4
sc.pl.umap(
        adata_annot_colon,
        color=["Ltb","Sell","Lef1","Nosip", "Ccr7"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
#tex
sc.pl.umap(
        adata_annot_colon,
        color=["Cxcr6","Gzmb","Havcr2","Ctla4", "Lag3","Cd274"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
#tcd8+ cells 
"Cd8a"

sc.pl.umap(
        adata_annot_colon,
        color=["Cd8a"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="inferno",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
sc.pp.neighbors(adata_annot_tumor)

In [ ]:
sc.tl.umap(adata_annot_tumor)

In [ ]:
sc.tl.leiden(adata_annot_tumor, resolution=0.7, key_added="leiden_3")

In [ ]:
sc.tl.leiden(adata_annot_tumor, key_added="leiden")

In [ ]:
sc.tl.leiden(adata_annot_tumor,resolution=0.2, key_added="leiden_2")

In [ ]:
sc.pl.umap(adata_annot_tumor, color = "leiden_3",legend_loc="on data")

In [ ]:
sc.pl.umap(adata_annot_tumor, color = "leiden_2",legend_loc="on data")

In [ ]:
sc.pl.umap(adata_annot_tumor, color = "manual_celltype_annotation")

In [ ]:
#tn 7.9.11.4, 5 left 
#leiden 3: 11,8,5,16,10,6,7
sc.pl.umap(
        adata_annot_tumor,
        color=["Ltb","Sell","Lef1","Nosip", "Ccr7"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
#tex 1,2,8,3 , 5 right  
# leiden3 9,13, 00,2,12,3,4,21,1,18,17
sc.pl.umap(
        adata_annot_tumor,
        color=["Cxcr6","Gzmb","Havcr2","Ctla4", "Lag3","Cd274"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
#tcd8+ cells 
"Cd8a"

sc.pl.umap(
        adata_annot_tumor,
        color=["Cd8a"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="inferno",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
#tisg
sc.pl.umap(
        adata_annot_tumor,
        color=["Isg15","Ifit3","Ifit1","Stat1", "Ifng"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="inferno",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
#trm
sc.pl.umap(
        adata_annot_tumor,
        color=["Itga1","Cpne7","Txnip","Jaml","Prf1","Gzmb"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
#trm
sc.pl.umap(
        adata_annot_tumor,
        color=["Prf1","Gzmb"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="inferno",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
#T cell regulatory
sc.pl.umap(
        adata_annot_tumor,
        color=["Foxp3","Ctla4","Il2ra"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="inferno",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
#9,13, 00,2,12,3,4,21,1,18,17 tex 
#11,8,5,16,10,6,7 tn

cl_annotation = { 
    "0": "Tex",
    "1": "Tex",
    "2": "Tex",
    "3":"Tex",
    "4":"Tex",
    "5":"Tn",
    "6":"Tn",
    "7":"Tn",
    "8":"Tn",
    "9":"Tex",
    "10":"Tn",
    "11":"Tn",
    "12":"Tex",
    "13":"Tex",
    "14":"Tex",
    "15":"Tex",
    "16":"Tn",
    "17":"Tex",
    "18":"Tex",
    "19":"Tex",
    "20":"Tex",
    "21":"Tex",
    "22":"Tex",
}

In [ ]:
adata_annot_tumor.obs["manual_celltype_annotation"] = adata_annot_tumor.obs.leiden_3.map(cl_annotation)

In [ ]:
adata_annot.X = adata_annot.layers["denoised"]

In [ ]:
scales_counts = sc.pp.normalize_total(adata, target_sum=None, inplace=False)
# log1p transform
adata.layers["log1p_norm_denoised"] = sc.pp.log1p(scales_counts["X"], copy=True)

In [ ]:
sc.pp.normalize_total(adata_annot)
sc.pp.log1p(adata_annot)

In [ ]:
sc.pl.umap(adata_solo, color = "leiden",legend_loc="on data")

In [ ]:
sc.pl.umap(adata_solo, color="leiden_2",legend_loc="on data")

In [ ]:
sc.pl.umap(adata_solo, color="leiden_3",legend_loc="on data")

In [ ]:
sc.pl.umap(adata_solo, color="leiden_3")

In [ ]:
#Sanity check
sc.pl.umap(adata_solo, color="is_doublet")

In [ ]:
sc.pl.umap(adata_annot, color=["manual_celltype_annotation"], save = "manual_celltype_annotation", title ="Single cell sorted CD8+ T cells")


In [ ]:
adata_annot

In [ ]:
adata_solo.obs.sample_id.unique()

In [ ]:
sc.pl.umap(adata_annot[adata_annot.obs["sample_id"].isin(["11mix1","11mix2"])], color="Cxcr3",gene_symbols ="symbol",  vmax=1, save="_cxcr3_11mix",  title = "Cxcr3 - Colon 11mix")
#sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["11mix1"])], color="Cxcr3")


In [ ]:
sc.pl.umap(adata_annot, color="Cxcr3",gene_symbols ="symbol",  vmax="1",  save = "_cxcr3_tumor_and_colon", title = "Cxcr3 - Colon and Tumor")
#sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["11mix1"])], color="Cxcr3")


In [ ]:
sc.pl.umap(adata_annot[adata_annot.obs["sample_id"].isin(["10mix-ICI1","10mix-ICI2","11mix-ICI1","11mix-ICI2","GF-ICI1","GF-ICI2","GF-ICI1-plus","GF-ICI2-plus"])], color="Cxcr3",gene_symbols ="symbol",  vmax="p99")
#sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["11mix1"])], color="Cxcr3")


In [ ]:
sc.pl.umap(adata_annot[adata_annot.obs["origin"].isin(["colon"])], color="Cxcr3",gene_symbols ="symbol",  save = "_cxcr3_colon", vmax=1, title = "Cxcr3 - Colon")
#sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["11mix1"])], color="Cxcr3")


In [ ]:
sc.pl.umap(adata_annot[adata_annot.obs["origin"].isin(["til"])], color="Cxcr3",gene_symbols ="symbol",    save = "_cxcr3_tumor",vmax=1, title = "Cxcr3 - Tumor")
#sc.pl.umap(adata_solo[adata_solo.obs["sample_id"].isin(["11mix1"])], color="Cxcr3")


Tnaive

Ltb","Sell","Lef1","Nosip

In [ ]:
#tn
sc.pl.umap(
        adata_annot,
        color=["Ltb","Sell","Lef1","Nosip", "Ccr7"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
#tn
sc.pl.umap(
        adata_annot,
        color=["Ltb","Sell","Lef1","Nosip", "Ccr7"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
sc.pl.umap(adata_annot, color=["Ltb","Sell","Lef1","Nosip","Ccr7"],gene_symbols="symbol", vmax=2)

In [ ]:
#tcd8+ cells 
"Cd8a"

sc.pl.umap(
        adata_annot,
        color=["Cd8a"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
#tex
sc.pl.umap(
        adata_annot,
        color=["Cxcr6","Gzmb","Havcr2","Ctla4", "Lag3","Cd274"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
sc.pl.umap(adata_annot, color=["Cxcr6","Gzmb","Havcr2","Ctla4", "Lag3"],gene_symbols="symbol", vmax=2)

In [ ]:
#Treg
sc.pl.umap(
        adata_annot,
        color=["Il2ra","Cd3d","Cd5","Cd14","Foxp3","Gata3","Batf","Cd226","Cxcr5","Bcl6"], #"Ccr8","Lag3" tumor infiltrated treg 
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
#Treg
sc.pl.umap(
        adata_annot,
        color=["Ikzf2","Ccr4","Entpd1","Il2ra","Itgae","Tnfrsf4"], #"Ccr8","Lag3" tumor infiltrated treg 
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
sc.pl.umap(adata_annot, color=["Il2ra","Cd3d","Cd5","Cd14"],gene_symbols="symbol", vmax=2)

In [ ]:
#tisg
sc.pl.umap(
        adata_annot,
        color=["Isg15","Ifit3","Ifit1","Stat1", "Ifng"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
sc.pl.umap(adata_annot, color=["Isg15","Ifit3","Ifit1","Stat1", "Ifng"],gene_symbols="symbol", vmax=2)

In [ ]:
#trm
sc.pl.umap(
        adata_annot,
        color=["Itga1","Cpne7","Txnip","Jaml"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
#trm
sc.pl.umap(
        adata_annot,
        color=["Cd44","Ccr7", "Cd28", "Cd69"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
sc.pl.umap(adata_annot, color=["Itga1","Cpne7","Txnip","Jaml"],gene_symbols="symbol", vmax=1)

In [ ]:
#tsen
sc.pl.umap(
        adata_annot,
        color=["Plcg2","Igkc","Klrk1","Slamf7"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
sc.pl.umap(adata_annot, color=["Plcg2","Igkc","Klrk1","Slamf7"],gene_symbols="symbol", vmax=1)

In [ ]:
#tsrt
sc.pl.umap(
        adata_annot,
        color=["Hspa1b", "Hspa1a","Dnajb1","Ubc"],
        vmin=0,
    gene_symbols="symbol",
        vmax="p99",  # set vmax to the 99th percentile of the gene count instead of the maximum, to prevent outliers from making expression in other cells invisible. Note that this can cause problems for extremely lowly expressed genes.
        sort_order=False,  # do not plot highest expression on top, to not get a biased view of the mean expression among cells
        frameon=False,
        cmap="Reds",  # or choose another color map e.g. from here: https://matplotlib.org/stable/tutorials/colors/colormaps.html
    )

In [ ]:
sc.pl.umap(adata_annot, color=["Ltb","Sell","Lef1","Nosip"],gene_symbols="symbol")

Tex

- Cxcr6
- Gzmb 
- Havcr2
- Ctla4

In [ ]:
sc.pl.umap(adata_solo, color=["Cxcr6","Gzmb","Havcr2","Ctla4"], gene_symbols="symbol")

Tisg
- Isg15
- Ifi44l
- Ifit3
- Ifit1

In [ ]:
sc.pl.umap(adata_solo, color=["Isg15","Ifit3","Ifit1","Stat1"],gene_symbols="symbol" )

Tn
- Ltb
- Sell
- Lef1
- Nosip

In [ ]:
sc.pl.umap(adata_solo, color=["Ltb","Sell","Lef1","Nosip"],gene_symbols="symbol" )

Trm
- Itga1
- Cpne7
- Txnip
- Jaml

In [ ]:
sc.pl.umap(adata_solo, color=["Itga1","Cpne7","Txnip","Jaml"],gene_symbols="symbol" )

Tsen
- Plcg2
- Igkc
- Klrk1
- Slamf7

In [ ]:
sc.pl.umap(adata_solo, color=["Plcg2","Igkc","Klrk1","Slamf7"],gene_symbols="symbol" )

Tsrt
- Hspa1b
- Hspa1a
- Dnajb1
- Ubc

In [ ]:
sc.pl.umap(adata_solo, color=["Hspa1b", "Hspa1a","Dnajb1","Ubc"],gene_symbols="symbol" )

In [ ]:
sc.pl.umap(adata_annot, color=["Tubb5", "Cdk1"],gene_symbols="symbol", cmap="Reds" )

In [ ]:
sc.pl.umap(adata_annot, color=['ENSMUSG00000022443',
 'ENSMUSG00000105361',
 'ENSMUSG00000029580',
 'ENSMUSG00000029635',
 'ENSMUSG00000021998',
 'ENSMUSG00000004980',
 'ENSMUSG00000030695',
 'ENSMUSG00000026728',
 'ENSMUSG00000092341',
 'ENSMUSG00000098178',
 'ENSMUSG00000059005'], vmax=3)

In [ ]:
sc.tl.rank_genes_groups(adata_annot, 'leiden_3', method='wilcoxon')

In [ ]:
adata_annot.var

In [ ]:
adata_annot_0 = adata_annot[adata_annot.obs["leiden"]=="0"].copy()

In [ ]:
dedf = sc.get.rank_genes_groups_df(adata_annot, group="10")

In [ ]:
sc.pl.umap(adata_solo, color="leiden_3")

In [ ]:
sc.pl.umap(adata_solo, color="leiden_3", legend_loc="on data")

Tn - Naive T cells
Tex - exhausted CD8+ T cell
Tisg - interferon (IFN) response T cell
Trm - Resident Memory T cells
Tsen - senescent-like T cells
Tstr - stress response T cell (identified a unique stress response state, TSTR, characterized by heat shock gene expression. TSTR cells are detectable in situ in the tumor microenvironment across various cancer types,)

In [ ]:
cl_annotation = {
    "0": "IFN T-cells",
    "1": "Naive T-cells",
    "2": "Exhausted T-cells",
    "3": "Naive T-cells",
    "4":"Senescent-like T-cells",
    "5":"Exhausted T-cells",
    "6":"Naive T-cells",
    "7":"Exhausted T-cells",
    "8":"IFN T-cells",
    "9":"Senescent-like T-cells",
    "10":"Exhausted T-cells",
    "11":"Undefined",
    "12":"Naive T-cells",
    "13":"Stress response T-cells",
}

In [ ]:
adata_solo.obs["manual_celltype_annotation"] = adata_solo.obs.leiden_3.map(cl_annotation)

In [ ]:
sc.pl.umap(adata_solo, color=["manual_celltype_annotation"],legend_loc="on data")

In [ ]:
sc.pl.umap(adata_annot, color=["manual_celltype_annotation"], save = "manual_celltype_annotation.png", title ="Single cell sorted CD8+ T cells")


In [ ]:
adata_solo.obs["manual_celltype_annotation"].value_counts().sort_index()

In [ ]:
marker_genes

In [ ]:
cl_annotation_1 = {"1":"Tn","3":"Tex","0":"Tsrt","5":"Undefined","2":"Tisg","6":"Undefined","4":"Undefined"}


In [ ]:
adata_solo.obs["manual_celltype_annotation_1"] = adata_solo.obs.leiden.map(cl_annotation_1)

In [ ]:
sc.pl.umap(adata_solo, color=["manual_celltype_annotation_1"],legend_loc="on data")

In [ ]:
sc.pl.umap(adata_solo, color=["manual_celltype_annotation_1"])

In [ ]:
adata_solo.var

In [ ]:
adata_annot

In [ ]:

sc.pl.violin(adata_annot, keys='n_genes', groupby='manual_celltype_annotation', rotation=90)

In [ ]:

sc.pl.violin(adata_annot, keys='n_genes', groupby='origin', rotation=90)

In [ ]:

sc.pl.violin(adata_annot, keys='n_genes', groupby='sample_id', rotation=90)

In [ ]:

sc.pl.violin(adata_annot, keys='total_counts', groupby='manual_celltype_annotation', rotation=90)

In [ ]:

sc.pl.violin(adata_annot, keys='total_counts', groupby='sample_id', rotation=90)

In [ ]:

sc.pl.violin(adata_annot, keys='total_counts', groupby='origin', rotation=90)

In [ ]:
adata_solo.var.index = adata_solo.var["query"]

In [ ]:
#markers = ["1Ab", "2Ab","3Ab","4Ab","CD103","CD44","CD69","ICOS","PD1"]
#sc.pl.dotplot(adata_solo, marker_genes, groupby='manual_celltype_annotation', dendrogram=True, gene_symbols="symbol")

In [ ]:
#sc.pl.matrixplot(adata_solo, marker_genes, groupby='manual_celltype_annotation', dendrogram=False, gene_symbols="symbol")

In [ ]:
adata_solo

In [ ]:
#sc.pl.heatmap(adata_solo, marker_genes, groupby='manual_celltype_annotation', swap_axes=True, gene_symbols="symbol", standard_scale="obs")


In [ ]:
marker_genes

In [ ]:
marker_list = ["1Ab", "2Ab","3Ab","4Ab","CD103","CD44","CD69","ICOS","PD1"]

In [ ]:
sc.pl.heatmap(adata_solo, marker_list, groupby='manual_celltype_annotation', swap_axes=True, log = True)


In [ ]:
marker_genes

In [ ]:
sc.pl.umap(adata_solo, color=["origin"],gene_symbols="symbol", save = "umap_origin.jpg", title ="Single cell sorted CD8+ T cells - Origin ")

In [ ]:
sc.pl.umap(adata_solo, color=["condition","origin"],gene_symbols="symbol" )
sc.pl.umap(adata_solo, color=["sample_id","ici_therapy"],gene_symbols="symbol" )

In [ ]:
sc.pl.umap(adata_solo, color=["CD103","CD44","CD69"] ,gene_symbols="symbol", save = "umap_marker_genes1.png")
sc.pl.umap(adata_solo, color=["ICOS","PD1"] ,gene_symbols="symbol",save = "umap_marker_genes2.png" )

In [ ]:
sc.pl.umap(adata_solo, color=["1Ab", "2Ab","3Ab","4Ab","CD103","CD44","CD69","ICOS","PD1"],gene_symbols="symbol" )

In [ ]:
sc.pl.umap(adata_solo, color=["Cxcr6", "Cxcr3","Cxcr4","Cxcr5"],gene_symbols="symbol" )

In [ ]:
adata_solo.var

In [ ]:
sc.pl.violin(adata_solo, keys='ENSMUSG00000050232', groupby='origin', rotation=90, gene_symbols="symbol") ##Cxcr3

In [ ]:
sc.pl.violin(adata_solo, keys='ENSMUSG00000050232', groupby='condition', rotation=90, gene_symbols="symbol") ##Cxcr3

In [ ]:
sc.pl.violin(adata_solo, keys='ENSMUSG00000048521', groupby='origin', rotation=90, gene_symbols="symbol") ##Cxcr6

In [ ]:
sc.pl.violin(adata_solo, keys='CD44', groupby='origin', rotation=90, gene_symbols="symbol") ##CD44

In [ ]:
sc.pl.violin(adata_solo, keys='PD1', groupby='origin', rotation=90, gene_symbols="symbol") ##PD1

In [ ]:
sc.pl.violin(adata_solo, keys='ENSMUSG00000048521', groupby='condition', rotation=90, gene_symbols="symbol") ##Cxcr6

In [ ]:
adata_annot.obs.columns

In [ ]:
sample10mix_h5ad=  sc.read_h5ad("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/scrna_seq_data/samples/adata_GF1.h5ad")


In [ ]:
sample10mix_h5ad.obs

In [ ]:
import pandas as pd

# Assuming adata_solo.obs.cell_type.value_counts() generates the cell type counts

# Get the cell type value counts
cell_type_counts = adata_annot.obs.sample_id.value_counts()

# Create a DataFrame from the cell type value counts
cell_type_df = pd.DataFrame(cell_type_counts).reset_index()

# Rename columns
cell_type_df.columns = ['cell_type', 'count']

# Display the resulting DataFrame
print(cell_type_df)

In [ ]:
cell_type_df.drop([0,1,2,3,4,5,6,7], axis=0, inplace=True)

In [ ]:
cell_type_df

In [ ]:
# initialize list of lists 
data = [['11mix', 7505], ['10mix', 6024], ['GF', 3132]] 
  
# Create the pandas DataFrame 
df = pd.DataFrame(data, columns=['cell_type', 'count']) 

In [ ]:
from matplotlib.ticker import MultipleLocator
import pandas as pd
import matplotlib.pyplot as plt

# Assuming cell_type_df is your DataFrame with cell type counts

# Create a bar plot using matplotlib
plt.figure(figsize=(10,4 ))  # Set the figure size

# Plot the data
plt.barh(df['cell_type'], df['count'], fill=False)

# Customize the plot
plt.xlabel('Number of cells')

plt.xticks(rotation=0)  # Rotate x-axis labels for better visibility

x_locator = MultipleLocator(base=1000)
plt.gca().xaxis.set_major_locator(x_locator)

# Show the plot
plt.tight_layout()  # Adjust layout for better spacing


plt.savefig("/home/kvalem/myScratch/projects/2021/microbial-metabolites/preprocessing/40_single-cell-sorted-cd8/40_scrna_seq/bin/notebooks/figures/colon_cell_number.pdf")
plt.show()

In [ ]:
genes = "/data/scratch/kvalem/projects/2021/microbial-metabolites/preprocessing/40_single-cell-sorted-cd8/40_scrna_seq/bin/notebooks/gene_markers.csv"

In [ ]:
genes_df = pd.read_csv(genes)

In [ ]:
genes_df = genes_df.drop_duplicates(subset=['symbol'])

In [ ]:
genes_df

### Automatic annotation

In [ ]:
dc.run_ora(
    mat=adata_solo,
    net=genes_df,
    source='cell_type',
    target='symbol',
    min_n=1,
    verbose=True
)

In [ ]:
acts = dc.get_acts(adata_solo, obsm_key='ora_estimate')

In [ ]:
acts_v = acts.X.ravel()
max_e = np.nanmax(acts_v[np.isfinite(acts_v)])
acts.X[~np.isfinite(acts.X)] = max_e

In [ ]:
acts